# Pre-processing MotionSense Dataset and Generate Views

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../../")

In [2]:
from pathlib import Path
from typing import List
import hashlib
import pandas as pd

from librep.datasets.har.motionsense import (
    RawMotionSense,
    RawMotionSenseIterator,
    MotionSenseDatasetGenerator
)
from librep.utils.dataset import PandasDatasetsIO

%matplotlib inline

2022-09-02 20:09:48.877338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-02 20:09:48.877358: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
dataset_dir = Path("../../data/datasets/MotionSense/A_DeviceMotion_data")
motionsense_dataset = RawMotionSense(dataset_dir, download=False)
motionsense_dataset

ValueError: invalid literal for int() with base 10: 'checkpoints'

In [ ]:
act_names = [motionsense_dataset.activity_names[i] for i in motionsense_dataset.activities]
act_names

In [ ]:
iterator = RawMotionSenseIterator(motionsense_dataset, users_to_select=None, activities_to_select=None)
iterator

In [ ]:
motionsense_v1 = MotionSenseDatasetGenerator(iterator, time_window=150, window_overlap=0)
motionsense_v1

In [ ]:
train, validation, test = motionsense_v1.create_datasets(
    train_size=0.7,
    validation_size=0.1,
    test_size=0.2,
    ensure_distinct_users_per_dataset=True,
    balance_samples=True,
    seed=0
)

In [ ]:
print(hashlib.sha1(pd.util.hash_pandas_object(train).values).hexdigest())
print(hashlib.sha1(pd.util.hash_pandas_object(validation).values).hexdigest())
print(hashlib.sha1(pd.util.hash_pandas_object(test).values).hexdigest())

In [ ]:
output_path = Path("../../data/views/MotionSense/balanced_view")

description = """# Balanced MotionSense

This view contains train, validation and test subsets in the following proportions:
- Train: 70% of samples
- Validation: 10% of samples
- Test: 20% of samples

After splits, the datasets were balanced in relation to the activity code column, that is, each subset have the same number of activitiy samples.

**NOTE**: Each subset contain samples from distinct users, that is, samples of one user belongs exclusivelly to one of three subsets.
"""
pandas_io = PandasDatasetsIO(output_path)
pandas_io

In [ ]:
pandas_io.save(
    train=train, 
    validation=validation, 
    test=test, 
    description=description
)